### Step 1: Import Essential Libraries

In [1]:
from google.cloud import bigquery
from scipy.stats.mstats import zscore
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import matplotlib as mpl
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest 
import seaborn as sns
import datetime as dt
from datetime import datetime,tzinfo
import scipy, json, csv, time, pytz
from pytz import timezone
import numpy as np
import pandas as pd
seed = 135
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os
os.listdir('../input/')

### First query for testing

> **Google BigQuery Bitcoin Blockchain Dataset** consist of two projects: Transactions and Blocks containing features like block_id, transaction_id, timestamp, input, output, etc., which updates every 10 minutes. [https://bigquery.cloud.google.com/dataset/bigquery-public-data:bitcoin_blockchain](http://)

In [2]:
#Connecting to Google datastore (use path to ur private key)
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="../input/gcp-bitcoin-project/Bitcoin Project-615d07137267.json"
client = bigquery.Client()

Let's see if we can query the whole btc chain for calculating somethign simple like the acceleration in the number of transactions along the whole history

![Crypto bitcoin schema](https://miro.medium.com/max/1400/1*Sl3uCSP1qTxB4iP-W5KL-w.png)

In [3]:
def query_to_df(query: str)->pd.DataFrame:
    query_job = client.query(query)
    # Waits for the query to finish
    iterator = query_job.result(timeout=30)
    rows = list(iterator)
    df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
    return df

# Let's see what it is in each table

#### Table Transactions

In [ ]:
query_x = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.transactions`
LIMIT 2
"""
df_x = query_to_df(query_x)
df_x.head(10)

#### Table Blocks

In [ ]:
query_x = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.blocks`
LIMIT 2
"""
df_x = query_to_df(query_x)
df_x.head(10)

#### Table Outputs

In [ ]:
query_x = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.outputs`
LIMIT 2
"""
df_x = query_to_df(query_x)
df_x.head(10)

#### Table Inputs

In [ ]:
query_x = """
SELECT *
FROM `bigquery-public-data.crypto_bitcoin.inputs`
LIMIT 2
"""
df_x = query_to_df(query_x)
df_x.head(10)

# Some Examples

In [ ]:
initial_date  = '2009-01-01'
final_date    = '2022-02-22'
# The query to get date, number of transactions from Google BigQuery bitcoin blockchain dataset 
# Select records from the last three years and group them with respect to date
query_1 = f"""
SELECT 
   DATE(timestamp) AS date,
   SUM(transaction_count) as num_transactions
     
FROM `bigquery-public-data.crypto_bitcoin.blocks`
GROUP BY date
HAVING date >= '{initial_date}'  AND date <= '{final_date}'
ORDER BY date
"""

query_job_1 = client.query(query_1)
# Waits for the query to finish
iterator_1 = query_job_1.result(timeout=30)
rows_1 = list(iterator_1)
df_1 = pd.DataFrame(data=[list(x.values()) for x in rows_1], columns=list(rows_1[0].keys()))

In [ ]:
print(df_1.head())
print(df_1.tail())

In [ ]:
plt.figure(figsize=(30,10))
sns.lineplot(x='date', y='num_transactions', data=df_1)

In [ ]:
df_1['acc'] = abs(df_1.num_transactions.diff() / df_1.date.diff().dt.total_seconds())

In [ ]:
plt.figure(figsize=(30,10))
sns.lineplot(x='date', y='acc', data=df_1[-720:]).set(title="Transactions acceleration")

In [ ]:
import pickle 

In [ ]:
with open('df_1.pickle', 'wb') as handle:
    pickle.dump(df_1, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Now something more serius for testing if there are some limitations in the querying the btc chain.
## Let's see if we can create and store a TXO dataframe

In [ ]:
# Start our query using WITH statement so we can define subqueries
QUERY = '''
WITH
'''

In [ ]:
# Append a subquery to our cumulative QUERY string
QUERY += '''
-- Outputs subquery: contains relevant information about a given output.
-- A TXO is created when it is an output of a transaction, so this contains
-- metadata about the TXO creation
output AS (
  SELECT
    transactions.HASH AS transaction_hash,
    transactions.block_number AS created_block_number,
    transactions.block_timestamp AS created_block_ts,
    outputs.index AS output_index,
    outputs.value AS output_value
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions` AS transactions,
    transactions.outputs AS outputs
    ),
'''

In [ ]:
QUERY += '''
-- Inputs subquery: contains relevant information about a given input.
-- A TXO is consumed when it is the input to a transaction, so this metadata
-- tells us about when a TXO is spent or destroyed
input AS (
  SELECT
    transactions.hash AS spending_transaction_hash,
    inputs.spent_transaction_hash AS spent_transaction_hash,
    transactions.block_number AS destroyed_block_number,
    transactions.block_timestamp AS destroyed_block_ts,
    inputs.spent_output_index,
    inputs.value AS input_value
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions` AS transactions,
    transactions.inputs AS inputs
    ),
'''

In [ ]:
print(QUERY)

In [ ]:
QUERY += '''
-- txo subquery: joins outputs to inputs so that we know when/if a TXO is spent.
txo AS (
  SELECT
    output.transaction_hash,
    output.created_block_number,
    DATETIME(output.created_block_ts) AS created_block_ts,
    -- Any field from the input table will be NULL if the TXO remains unspent.
    input.spending_transaction_hash,
    input.spent_transaction_hash,
    input.destroyed_block_number,
    DATETIME(input.destroyed_block_ts) AS destroyed_block_ts,
    output.output_value
  FROM
    output
  -- Use Left Join, as not all outputs will be linked as inputs in future transactions if they remain unspent.
  LEFT JOIN
    input
  ON
    -- Join an output to a future input based on the output transaction hash
    -- matching the spent transaction hash of the input
    output.transaction_hash = input.spent_transaction_hash
    -- Also make sure the output index matches within the transaction hash
    AND output.output_index = input.spent_output_index
  )
'''

In [ ]:
print(QUERY)

In [ ]:
# this query works but the result is so big that it can't be used in Kaggle server.
#query_TXOs = client.query(QUERY + "SELECT * FROM txo")
# Waits for the query to finish
#iterator_TXOs = query_TXOs.result()
#rows_TXOs = list(iterator_TXOs)
#df_TXOs = pd.DataFrame(data=[list(x.values()) for x in rows_TXOs], columns=list(rows_TXOs[0].keys()))

In [ ]:
#with open('df_TXOs.pickle', 'wb') as handle:
#    pickle.dump(df_TXOs, handle, protocol=pickle.HIGHEST_PROTOCOL)
#compression_opts = dict(method='zip',archive_name='df_TXOs.csv')  
#df.to_csv('df_TXOs.zip', index=False,compression=compression_opts) 

## The next query will help us to create the HOLD HEATMAP

In [ ]:
QUERY += '''
-- blocks subquery: for each date get the final block for that date
blocks AS (
  SELECT
    DATE(timestamp) AS date,
    -- Get last block per day
    MAX(number) AS block_number,
    MAX(DATETIME(timestamp)) AS block_ts
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks`
  GROUP BY
    date)
'''

In [ ]:
QUERY += '''
-- final data aggregation query: join txo with blocks, keeping only txo 
-- that were created and unspent as of that block, then bucket the txo
-- by age and sum the txo value per bucket per that day
SELECT
  -- Time series metadata
  blocks.date AS date,
  blocks.block_number AS block_number,
  blocks.block_ts AS block_ts,
  
-- BTC Value Weighting
  -- Total UTXO value on that date
  SUM(txo.output_value) AS total_utxo_value,
  -- Our HODL Waves buckets, counting value of UTXO
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 1, txo.output_value, 0)) AS utxo_value_under_1d,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 1
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 7,
         txo.output_value, 0)) AS utxo_value_1d_1w,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 7
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28,
         txo.output_value, 0)) AS utxo_value_1w_1m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 3,
         txo.output_value, 0)) AS utxo_value_1m_3m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 3
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 6,
         txo.output_value, 0)) AS utxo_value_3m_6m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 6
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12,
         txo.output_value, 0)) AS utxo_value_6m_12m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 18,
         txo.output_value, 0)) AS utxo_value_12m_18m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 18
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 24,
         txo.output_value, 0)) AS utxo_value_18m_24m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 2
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12 * 3,
         txo.output_value, 0)) AS utxo_value_2y_3y,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 3
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12 * 5,
         txo.output_value, 0)) AS utxo_value_3y_5y,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 5
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12 * 8,
         txo.output_value, 0)) AS utxo_value_5y_8y,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 8,
         txo.output_value, 0)) AS utxo_value_greater_8y,

-- Flat Weighting
  -- Total UTXO count on that date
  SUM(1) AS total_utxo_count,
  -- Our HODL Waves buckets, counting number of UTXO
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 1, 1, 0)) AS utxo_count_under_1d,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 1
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 7,
         1, 0)) AS utxo_count_1d_1w,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 7
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28,
         1, 0)) AS utxo_count_1w_1m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 3,
         1, 0)) AS utxo_count_1m_3m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 3
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 6,
         1, 0)) AS utxo_count_3m_6m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 6
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12,
         1, 0)) AS utxo_count_6m_12m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 18,
         1, 0)) AS utxo_count_12m_18m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 18
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 24,
         1, 0)) AS utxo_count_18m_24m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 2
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12 * 3,
         1, 0)) AS utxo_count_2y_3y,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 3
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12 * 5,
         1, 0)) AS utxo_count_3y_5y,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 5
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12 * 8,
         1, 0)) AS utxo_count_5y_8y,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 8,
         1, 0)) AS utxo_count_greater_8y,

-- Flat weighting, filtered
  -- Total UTXO count on that date (> 0.01 BTC)
  SUM(IF(txo.output_value / 100000000 > 0.01, 1, 0)) AS total_utxo_count_filter,
  -- Our HODL Waves buckets, counting number of UTXO (> 0.01 BTC)
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 1
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_under_1d,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 1
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 7
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_1d_1w,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 7
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_1w_1m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 3
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_1m_3m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 3
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 6
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_3m_6m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 6
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_6m_12m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 18
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_12m_18m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 18
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 24
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_18m_24m,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 2
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12 * 3
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_2y_3y,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 3
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12 * 5
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_3y_5y,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 5
         AND DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) < 28 * 12 * 8
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_5y_8y,
  SUM(IF(DATETIME_DIFF(blocks.block_ts, txo.created_block_ts, DAY) >= 28 * 12 * 8
         AND txo.output_value / 100000000 >= 0.01,
         1, 0)) AS utxo_count_filter_greater_8y
FROM
  blocks
CROSS JOIN
  txo
WHERE
  -- Only include transactions that were created on or after the given block
  blocks.block_number >= txo.created_block_number
  -- Only include transactions there were unspent as of the given block
  AND (
    -- Transactions that are spent after the given block, so they are included
    blocks.block_number < txo.destroyed_block_number 
    -- Transactions that are never spent, so they are included
    OR txo.destroyed_block_number IS NULL)
GROUP BY
  date, block_number, block_ts
ORDER BY
  date ASC;
'''

In [ ]:
print(QUERY)

## Now we will check the addresses

In [ ]:
QUERY = ''
QUERY += '''
WITH

output AS (
  SELECT
      tx.hash AS transaction_hash
    , tx.block_timestamp AS block_ts
    , tx.block_number AS block_number
    , addresses AS address
    , outputs.value AS value
FROM    `bigquery-public-data.crypto_bitcoin.transactions` AS tx,
    tx.outputs AS outputs,
    UNNEST(outputs.addresses) AS addresses
-- WHERE    block_timestamp_month >= "2021-05-01"
),

address_stats AS (
SELECT
    address
  , MIN(block_number) AS first_block_used
  , COUNT(DISTINCT transaction_hash) AS num_txs
FROM output
GROUP BY 1)

SELECT
    DATE(block_ts) AS date

  , COUNT(output.address) AS address_count
  , SUM(IF(address_stats.first_block_used = block_number, 1, 0)) AS new_address_count
  , SUM(IF(address_stats.first_block_used < block_number AND num_txs > 1, 1, 0)) AS reused_address_count
  , SAFE_DIVIDE(SUM(IF(address_stats.first_block_used < block_number AND num_txs > 1, 1, 0)), COUNT(output.address)) AS pct_reused_count

  , COUNT(DISTINCT output.address) AS address_ucount
  , COUNT(DISTINCT IF(address_stats.first_block_used = block_number, output.address, Null)) AS new_address_ucount
  , COUNT(DISTINCT IF(address_stats.first_block_used < block_number AND num_txs > 1, output.address, Null)) AS reused_address_ucount
  , SAFE_DIVIDE(COUNT(DISTINCT IF(address_stats.first_block_used < block_number AND num_txs > 1, output.address, Null)), COUNT(DISTINCT output.address)) AS pct_reused_ucount

  , SUM(output.value) AS address_value
  , SUM(IF(address_stats.first_block_used = block_number, output.value, 0)) AS new_address_value
  , SUM(IF(address_stats.first_block_used < block_number AND num_txs > 1, output.value, 0)) AS reused_address_value
  , SAFE_DIVIDE(SUM(IF(address_stats.first_block_used < block_number AND num_txs > 1, output.value, 0)), SUM(output.value)) AS pct_reused_value

  , SUM(IF(num_txs <= 100, 1, 0)) AS address_count_small
  , SUM(IF(address_stats.first_block_used = block_number AND num_txs <= 100, 1, 0)) AS new_address_count_small
  , SUM(IF(address_stats.first_block_used < block_number AND num_txs > 1 AND num_txs <= 100, 1, 0)) AS reused_address_count_small
  , SAFE_DIVIDE(SUM(IF(address_stats.first_block_used < block_number AND num_txs > 1 AND num_txs <= 100, 1, 0)), SUM(IF(num_txs <= 100, 1, 0))) AS pct_reused_count_small

  , COUNT(DISTINCT IF(num_txs <= 100, output.address, NULL)) AS address_ucount_small
  , COUNT(DISTINCT IF(address_stats.first_block_used = block_number AND num_txs <= 100, output.address, Null)) AS new_address_ucount_small
  , COUNT(DISTINCT IF(address_stats.first_block_used < block_number AND num_txs > 1 AND num_txs <= 100, output.address, Null)) AS reused_address_ucount_small
  , SAFE_DIVIDE(COUNT(DISTINCT IF(address_stats.first_block_used < block_number AND num_txs > 1 AND num_txs <= 100, output.address, Null)), COUNT(DISTINCT IF(num_txs <= 100, output.address, NULL))) AS pct_reused_ucount_small

  , SUM(IF(num_txs <= 100, output.value, 0)) AS address_value_small
  , SUM(IF(address_stats.first_block_used = block_number AND num_txs <= 100, output.value, 0)) AS new_address_value_small
  , SUM(IF(address_stats.first_block_used < block_number AND num_txs > 1 AND num_txs <= 100, output.value, 0)) AS reused_address_value_small
  , SAFE_DIVIDE(SUM(IF(address_stats.first_block_used < block_number AND num_txs > 1 AND num_txs <= 100, output.value, 0)), SUM(IF(num_txs <= 100, output.value, 0))) AS pct_reused_value_small
FROM output
LEFT JOIN address_stats
ON output.address = address_stats.address
GROUP BY 1
ORDER BY 1 ASC
'''

In [ ]:
query_addresses= client.query(QUERY)
# Waits for the query to finish
iterator_addresses = query_addresses.result()
rows_addresses = list(iterator_addresses)
df_addresses = pd.DataFrame(data=[list(x.values()) for x in rows_addresses], columns=list(rows_addresses[0].keys()))

In [ ]:
with open('df_addresses.pickle', 'wb') as handle:
    pickle.dump(df_addresses, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
compression_opts = dict(method='zip',archive_name='df_addresses.csv')  
df_addresses.to_csv('df_addresses.zip', index=False, compression=compression_opts) 